<a href="https://colab.research.google.com/github/desireedisco/MS_AI_Machine_Learning_Unsupervised/blob/main/Book_Recommender_Preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**This notebook is designed to preprocess the data by cleaning and simplifying the data to get ready for topic labeling for book recommendation based on content filtering.**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**The bookcrossing-dataset is located here:**

**https://www.kaggle.com/datasets/ruchi798/bookcrossing-dataset/data**

In [ ]:
import kagglehub
ruchi798_bookcrossing_dataset_path = kagglehub.dataset_download('ruchi798/bookcrossing-dataset')

Using Colab cache for faster access to the 'bookcrossing-dataset' dataset.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

import warnings
warnings.filterwarnings("ignore")

**Data Description**

The dataset contains information on 278,858 users with demographic information, 1,149,780 ratings that are a combination of explicit and implicit ratings, and 271,379 books. The dataset contains 4 different csv files: BX-Users.csv, BX_Books.csv, BX-Book-Ratings.csv, and Preprocessed_data.csv. The preprocess_data.csv is the only place where the subject tag and book summary can be found. I primarily use the preprocess_data file for my book information.


**Data Cleaning Plan**

*   Check for null values, remove duplicates, drop unnecessary data
*   Make sure books have a subject tag and text summary
*   Check for language of summary
*   Check for number of ratings per user - remove users with less than 20 ratings
*   Check for number of ratings per book - remove books with less than 20 ratings


In [ ]:
# load the data
users=pd.read_csv(ruchi798_bookcrossing_dataset_path + '/Book reviews/Book reviews/BX-Users.csv', sep=";", encoding='latin-1')
combo = pd.read_csv(ruchi798_bookcrossing_dataset_path + '/Books Data with Category Language and Summary/Preprocessed_data.csv', sep=",", encoding='latin-1', on_bad_lines='skip', engine='python')
books=pd.read_csv(ruchi798_bookcrossing_dataset_path + '/Book reviews/Book reviews/BX_Books.csv', sep=";", encoding='latin-1')
ratings=pd.read_csv(ruchi798_bookcrossing_dataset_path + '/Book reviews/Book reviews/BX-Book-Ratings.csv', sep=";", encoding='latin-1')

print("Books Data:    ", books.shape)
print("Users Data:    ", users.shape)
print("Books-ratings: ", ratings.shape)

Books Data:     (271379, 8)
Users Data:     (278858, 3)
Books-ratings:  (1149780, 3)


## **Clean books csv**

**Look at the data**

In [ ]:
books.info()
books.drop(columns=['Image-URL-S', 'Image-URL-M', 'Image-URL-L'], inplace=True)
books['ISBN'] = books['ISBN'].str.upper()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271379 entries, 0 to 271378
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ISBN                 271379 non-null  object
 1   Book-Title           271379 non-null  object
 2   Book-Author          271377 non-null  object
 3   Year-Of-Publication  271379 non-null  int64 
 4   Publisher            271377 non-null  object
 5   Image-URL-S          271379 non-null  object
 6   Image-URL-M          271379 non-null  object
 7   Image-URL-L          271379 non-null  object
dtypes: int64(1), object(7)
memory usage: 16.6+ MB


In [ ]:
print(books.describe(include=['object']))
print(books.head())

              ISBN      Book-Title      Book-Author  Publisher
count       271379          271379           271377     271377
unique      271065          242154           102027      16806
top     080410753X  Selected Poems  Agatha Christie  Harlequin
freq             2              27              632       7536
         ISBN                                         Book-Title  \
0  0195153448                                Classical Mythology   
1  0002005018                                       Clara Callan   
2  0060973129                               Decision in Normandy   
3  0374157065  Flu: The Story of the Great Influenza Pandemic...   
4  0393045218                             The Mummies of Urumchi   

            Book-Author  Year-Of-Publication                Publisher  
0    Mark P. O. Morford                 2002  Oxford University Press  
1  Richard Bruce Wright                 2001    HarperFlamingo Canada  
2          Carlo D'Este                 1991          Harper

**Remove duplicates**

In [ ]:
# check for duplicates in book-title with the same ISBN
duplicate_isbn = books[books.duplicated(subset=['ISBN', 'Book-Title'], keep=False)].sort_values(by=['ISBN'])
print(len(duplicate_isbn))
print(duplicate_isbn)

626
              ISBN                                 Book-Title     Book-Author  \
246913  000225669X                One Thousand Chestnut Trees      Mira Stout   
239830  000225669X                One Thousand Chestnut Trees      Mira Stout   
199492  000648302X                           Before and After  Matthew Thomas   
75013   000648302X                           Before and After  Matthew Thomas   
54573   000649840X                              Angelas Ashes   Frank Mccourt   
...            ...                                        ...             ...   
238019  349922271X  Illuminatus 01. Das Auge in der Pyramide.     Robert Shea   
53006   374661922X                 Der Sterne TennisbÃ?Â¤lle.     Stephen Fry   
205536  374661922X                 Der Sterne TennisbÃ?Â¤lle.     Stephen Fry   
97667   887641486X                    I Giorni Dell'abbandono  Elena Ferrante   
543     887641486X                    I Giorni Dell'abbandono  Elena Ferrante   

        Year-Of-Publica

In [ ]:
# droup duplicates
books.drop_duplicates(subset=['ISBN', 'Book-Title'], keep='first', inplace=True)
books.reset_index(drop=True, inplace=True)

In [ ]:
# special condition
drop_dup_condition = (books['ISBN'] == '051513628X') & (books['Book-Title'] == 'Key of Light (Key Trilogy (Paperback))')
books = books[~drop_dup_condition]
books.reset_index(drop=True, inplace=True)

In [ ]:
# no more duplicates of title and ISBN - there are still duplicates of title but they do not have the same ISBN
duplicate_isbn = books[books.duplicated(subset=['ISBN'], keep=False)].sort_values(by=['ISBN'])
print(len(duplicate_isbn))
print(duplicate_isbn)

0
Empty DataFrame
Columns: [ISBN, Book-Title, Book-Author, Year-Of-Publication, Publisher]
Index: []


In [ ]:
print(books.describe(include=['object']))

              ISBN      Book-Title      Book-Author  Publisher
count       271065          271065           271063     271063
unique      271065          242153           102027      16806
top     0767409752  Selected Poems  Agatha Christie  Harlequin
freq             1              27              631       7533


In [ ]:
books.isnull().sum()

,0
ISBN,0
Book-Title,0
Book-Author,2
Year-Of-Publication,0
Publisher,2


**I am not going to worry about the NaN values right now**

In [ ]:
books.loc[books['Book-Author'].isnull(),:]

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher
117967,0751352497,A+ Quiz Masters:01 Earth,NaN,1999,Dorling Kindersley
187545,9627982032,The Credit Suisse Guide to Managing Your Perso...,NaN,1995,Edinburgh Financial Publishing


In [ ]:
books.loc[books['Publisher'].isnull(),:]

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher
128817,193169656X,Tyrant Moon,Elaine Corvidae,2002,NaN
128964,1931696993,Finders Keepers,Linnea Sinclair,2001,NaN


## **Clean preprocessed csv**

**If I was looking at the location of users I would save the location infomation because it is already separated out, but I am more focused on topics and content based recommendations for this project.**

In [ ]:
combo.info()
combo.drop(columns=['location', 'city', 'state', 'img_s', 'img_m', 'img_l'], inplace=True)
combo['isbn'] = combo['isbn'].str.upper()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1031175 entries, 0 to 1031174
Data columns (total 19 columns):
 #   Column               Non-Null Count    Dtype  
---  ------               --------------    -----  
 0   Unnamed: 0           1031175 non-null  int64  
 1   user_id              1031175 non-null  int64  
 2   location             1031175 non-null  object 
 3   age                  1031175 non-null  float64
 4   isbn                 1031175 non-null  object 
 5   rating               1031175 non-null  int64  
 6   book_title           1031175 non-null  object 
 7   book_author          1031174 non-null  object 
 8   year_of_publication  1031175 non-null  float64
 9   publisher            1031175 non-null  object 
 10  img_s                1031175 non-null  object 
 11  img_m                1031175 non-null  object 
 12  img_l                1031175 non-null  object 
 13  Summary              1031175 non-null  object 
 14  Language             1031175 non-null  object 
 15

In [ ]:
print(combo.head())

   Unnamed: 0  user_id      age        isbn  rating           book_title  \
0           0        2  18.0000  0195153448       0  Classical Mythology   
1           1        8  34.7439  0002005018       5         Clara Callan   
2           2    11400  49.0000  0002005018       0         Clara Callan   
3           3    11676  34.7439  0002005018       8         Clara Callan   
4           4    41385  34.7439  0002005018       0         Clara Callan   

            book_author  year_of_publication                publisher  \
0    Mark P. O. Morford               2002.0  Oxford University Press   
1  Richard Bruce Wright               2001.0    HarperFlamingo Canada   
2  Richard Bruce Wright               2001.0    HarperFlamingo Canada   
3  Richard Bruce Wright               2001.0    HarperFlamingo Canada   
4  Richard Bruce Wright               2001.0    HarperFlamingo Canada   

                                             Summary Language  \
0  Provides an introduction to classica

**Replace the '9' in category and summary columns**

In [ ]:
combo['Category'] = combo['Category'].replace('9', np.nan)
combo['Summary'] = combo['Summary'].replace('9', np.nan)

**Fill nans will information from duplicates with the infomation needed. This is so it will not matter which duplicate we delete.**

In [ ]:
# fill NaNs in 'Category' based on the non-NaN value within each group of (['isbn', 'book_title']) duplicates
combo['Category'] = combo.groupby(['isbn', 'book_title'])['Category'].transform(lambda x: x.fillna(x.dropna().iloc[0] if not x.dropna().empty else np.nan))
combo['Summary'] = combo.groupby(['isbn', 'book_title'])['Summary'].transform(lambda x: x.fillna(x.dropna().iloc[0] if not x.dropna().empty else np.nan))

In [ ]:
print(combo[combo['Category'].isnull()])

         Unnamed: 0  user_id      age        isbn  rating  \
68               68        8  34.7439  0671870432       0   
69               69    11676  34.7439  0671870432       8   
70               70    24539  34.7439  0671870432       0   
80               80        8  34.7439  0887841740       5   
81               81    92861  34.7439  0887841740       0   
...             ...      ...      ...         ...     ...   
1031161     1031161   278843  28.0000  0373262388       0   
1031163     1031163   278843  28.0000  0743525493       7   
1031167     1031167   278851  33.0000  0028630289       0   
1031172     1031172   278851  33.0000  0884159221       7   
1031174     1031174   278851  33.0000  1569661057      10   

                                                book_title    book_author  \
68                                         PLEADING GUILTY    Scott Turow   
69                                         PLEADING GUILTY    Scott Turow   
70                                  

**Grouping by book info**

In [ ]:
book_info = combo.groupby(['isbn', 'book_title', 'Category','Summary']).agg(rating_count=('rating','count'), rating_avg=('rating','mean')).reset_index().sort_values(by=['book_title'])

In [ ]:
print(len(book_info))

138548


In [ ]:
book_info.head(10)

,isbn,book_title,Category,Summary,rating_count,rating_avg
69611,0590567330,A Light in the Storm: The Civil War Diary of ...,['Juvenile Fiction'],Captures the emotions of a young Union girl wh...,4,2.25
16572,0310232546,"Ask Lily (Young Women of Faith: Lily Series, ...",['Juvenile Fiction'],"When Lily enters the seventh grade, she starts...",1,8.00
116713,0962295701,Beyond IBM: Leadership Marketing and Finance ...,['Business & Economics'],Some of the world&#39;s most successful entrep...,1,0.00
46429,0439188970,Clifford Visita El Hospital (Clifford El Gran...,['Juvenile Fiction'],Clifford acompa a a Emily Elizabeth a visitar ...,1,0.00
43348,0399151788,Dark Justice,['Fiction'],An attempt on the life of the U.S. president d...,1,10.00
123762,1566869250,Final Fantasy Anthology: Official Strategy Gu...,['Games'],This strategy guide includes complete walkthro...,4,5.00
103542,082177350X,Flight of Fancy: American Heiresses (Zebra Ba...,['Fiction'],"Arriving in Paris with her newly rich family, ...",2,4.00
23994,0345407709,Garfield Bigger and Better (Garfield (Numbere...,['Humor'],A collection of cartoons and comic strips feat...,2,3.50
79028,0679732578,Good Wives: Image and Reality in the Lives of...,['History'],Examines the lives of women in northern New En...,10,3.20
111873,0889652015,Harry Potter and the Bible: The Menace Behind...,['Religion'],Examines J.K. Rowling&#39;s &quot;Harry Potter...,1,0.00


In [ ]:
# special condition
drop_dup_condition = (book_info['isbn'] == '051513628X') & (book_info['book_title'] == 'Key of Light (Key Trilogy (Paperback))')
book_info = book_info[~drop_dup_condition]
book_info.reset_index(drop=True, inplace=True)

In [ ]:
duplicate_isbn = book_info[book_info.duplicated(subset=['isbn'], keep=False)]
print(duplicate_isbn)

Empty DataFrame
Columns: [isbn, book_title, Category, Summary, rating_count, rating_avg]
Index: []


In [ ]:
list_of_book_title_w_cat =book_info.groupby(['book_title', 'Category'])['rating_count'].sum().reset_index().sort_values(by=['book_title'])
list_of_book_title_w_cat.head(5)

,book_title,Category,rating_count
0,A Light in the Storm: The Civil War Diary of ...,['Juvenile Fiction'],4
1,"Ask Lily (Young Women of Faith: Lily Series, ...",['Juvenile Fiction'],1
2,Beyond IBM: Leadership Marketing and Finance ...,['Business & Economics'],1
3,Clifford Visita El Hospital (Clifford El Gran...,['Juvenile Fiction'],1
4,Dark Justice,['Fiction'],1


**The are duplicate titles with different category tags**

In [ ]:
duplicate_title = list_of_book_title_w_cat[list_of_book_title_w_cat.duplicated(subset=['book_title'], keep=False)].sort_values(by=['book_title','Category'])
print(duplicate_title.head(20))

                                            book_title  \
95                                      10 Lb. Penalty   
96                                      10 Lb. Penalty   
228                                     101 Dalmatians   
229                                     101 Dalmatians   
247                                   101 Famous Poems   
248                                   101 Famous Poems   
269                                101 President Jokes   
270                                101 President Jokes   
347                     1066: The Year of the Conquest   
348                     1066: The Year of the Conquest   
448                                               1984   
449                                               1984   
450                                               1984   
453                                               1988   
454                                               1988   
485                       20,000 Leagues Under the Sea   
486           

## Number of books after cleaning 138,547

In [ ]:
# just a number check
len(book_info)

138547

## **Make master book dataframe**

In [ ]:
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton & Company


In [ ]:
print(len(books))
master_book_df = pd.merge(books,book_info[['isbn', 'Category', 'Summary', 'rating_count']], left_on='ISBN', right_on='isbn', how='left')
print(len(master_book_df))

271065
271065


In [ ]:
master_book_df.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,isbn,Category,Summary,rating_count
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,0195153448,['Social Science'],Provides an introduction to classical myths pl...,1.0
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,0002005018,['Actresses'],"In a small town in Canada, Clara Callan reluct...",14.0
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,0060973129,['1940-1949'],"Here, for the first time in paperback, is an o...",3.0
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,0374157065,['Medical'],"Describes the great flu epidemic of 1918, an o...",11.0
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton & Company,0393045218,['Design'],A look at the incredibly well-preserved ancien...,1.0


In [ ]:
# drop books with no categories
master_book_df.dropna(subset=['Category'], inplace=True)
master_book_df.reset_index(drop=True, inplace=True)
print(len(master_book_df))

138547


In [ ]:
# make rating_count an int again
master_book_df['rating_count'] = master_book_df['rating_count'].astype(int)

In [ ]:
# fill na on publisher
master_book_df['Publisher'] = master_book_df['Publisher'].fillna('Unknown')

In [ ]:
master_book_df.isnull().sum()

,0
ISBN,0
Book-Title,0
Book-Author,0
Year-Of-Publication,0
Publisher,0
isbn,0
Category,0
Summary,0
rating_count,0


## **Clean up category, summary, and check language**

###Clean Category

In [ ]:
def clean_category(text):
  category = ''

  square_bracket_content = re.findall(r'\[(.*?)\]', text)

  if square_bracket_content:
    # re.findall returns a list, e.g., ["'Social Science', 'History'"]
    # We need to take the first element of this list, which is the string containing categories.
    content_str = square_bracket_content[0]

    # remove quotes from the category string
    content_str = content_str.replace("'", '').replace('"', '').replace('.','').strip().lower()

  return content_str

In [ ]:
master_book_df.loc[:,'cat_clean'] = master_book_df.loc[:,'Category'].apply(clean_category)

In [ ]:
cat_unique = pd.DataFrame(master_book_df['cat_clean'].unique())
print(cat_unique)

                                   0
0                     social science
1                          actresses
2                          1940-1949
3                            medical
4                             design
...                              ...
6113                   cardiologists
6114                     exodus, the
6115            cross-country skiing
6116  faschismus - massenpsychologie
6117               signal processing

[6118 rows x 1 columns]


In [ ]:
all_categories = [category for category in master_book_df['cat_clean']]
unique_category = pd.Series(all_categories).unique()
print(unique_category.tolist())
print(len(unique_category))

['social science', 'actresses', '1940-1949', 'medical', 'design', 'fiction', 'history', 'nature', 'humor', 'cooking', 'reference', 'juvenile fiction', 'canadian fiction', 'health & fitness', 'control (psychology)', 'cholistan desert (pakistan)', 'science', 'political ethics', 'conduct of life', 'horses', 'political science', 'brothers', 'baggins, frodo (fictitious character)', 'biography & autobiography', 'business & economics', 'americans', 'infanticide', 'cabala', 'literary criticism', 'self-help', 'american fiction', 'humorous stories', 'diary fiction', 'religion', 'german language', 'bloom, leopold (fictitious character)', 'detective and mystery stories', 'gargoyles (fictitious characters)', 'oglala indians', 'england', 'comic books, strips, etc', 'comics & graphic novels', 'language arts & disciplines', 'aerospace engineers', 'greek poetry', 'juvenile nonfiction', 'gay men', 'orphans', 'family problems', 'computers', 'american literature', 'popular music', 'businessmen', 'psycholo

In [ ]:
# calculate value counts
counts = pd.Series(all_categories).value_counts()
counts_below_20 = counts[counts < 20]

print(counts_below_20.head(20))
print(len(counts_below_20))

chicago (ill)           19
biography               19
food                    19
books and reading       19
brazil                  19
jews                    19
high school students    19
colorado                18
cancer                  18
german language         18
elephants               18
beauty, personal        18
farm life               18
ballet                  18
devotional calendars    18
alcoholics              18
comedians               18
electronic books        18
christianity            18
businessmen             18
Name: count, dtype: int64
5882


###Clean Summary

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [ ]:
# add additional stop words that aren't in the nltk library list
add_stop_words = {'released', 'available', 'purchase', 'edition', 'book', 'bestselling', 'fiction', 'published', 'novel'}
print(add_stop_words)

# load the nltk library stop word list
stop_words = set(stopwords.words('english'))

# combine the 2 lists - don't really need this now because I did not add a lot of additional stop words but leaving in as place holder
stop_words = stop_words.union(add_stop_words)

# display stop words
print(stop_words)
print('said' in stop_words)

{'edition', 'book', 'released', 'fiction', 'bestselling', 'novel', 'purchase', 'available', 'published'}
{'hers', 'book', 'or', "needn't", 'because', 'out', 'of', 'd', 'up', 'until', 'against', 'most', 've', 'the', 'a', 'wasn', "it'd", "you'll", 'it', 'published', 'ma', "it's", 'an', 'having', 'edition', 'so', 'to', 'once', "they'll", 'were', 'should', "mightn't", "we've", 'his', 'in', 's', 'only', 'after', 'don', "he'd", 'its', 'being', 'her', 'they', 'does', 'too', "it'll", 'this', "you'd", 'had', 'novel', "wouldn't", 'with', 'didn', "doesn't", 'what', 're', "i'm", "i'd", 'themselves', 'mightn', 'that', "weren't", 'from', 'be', "hadn't", 'doesn', 'was', 'while', "won't", 'my', 'some', 'any', 'above', 'll', 'have', 'wouldn', 'nor', 'over', 'yours', 'than', 'theirs', 'your', 'did', "i've", "you're", 'under', 'do', "don't", 'very', 'then', "she's", "we're", 'weren', 'couldn', 'through', 'we', 'doing', 'mustn', 'them', 'm', 'isn', 'him', 't', 'fiction', 'been', 'yourself', 'down', 'how',

In [ ]:
def clean_summary(text):
  text = text.lower()

  # remove punctuation
  punct_pattern = r'[^\w\s]+'
  text = re.sub(punct_pattern, '', text)

  # remove numbers
  num_pattern = r'[0-9]+'
  text = re.sub(num_pattern, '', text)

  # remove spaces
  text = re.sub(' +', ' ', text)

  # tokenize text
  tokens = word_tokenize(text)

  # remove stop words and join back to text string
  #stop_words = stopwords.words('english')
  stop_words_removed = [word for word in tokens if word not in stop_words and len(word)>2]

  clean_text = ' '.join(stop_words_removed)
  return clean_text

In [ ]:
# clean text and remove stop words
master_book_df.loc[:,'summary_clean'] = master_book_df.loc[:,'Summary'].apply(clean_summary)

In [ ]:
master_book_df.loc[:,'len_sum'] = master_book_df.loc[:,'summary_clean'].str.len()

In [ ]:
master_book_df.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,isbn,Category,Summary,rating_count,cat_clean,summary_clean,len_sum
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,0195153448,['Social Science'],Provides an introduction to classical myths pl...,1,social science,provides introduction classical myths placing ...,179
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,0002005018,['Actresses'],"In a small town in Canada, Clara Callan reluct...",14,actresses,small town canada clara callan reluctantly tak...,81
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,0060973129,['1940-1949'],"Here, for the first time in paperback, is an o...",3,1940-1949,first time paperback outstanding military hist...,171
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,0374157065,['Medical'],"Describes the great flu epidemic of 1918, an o...",11,medical,describes great flu epidemic outbreak killed f...,171
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton & Company,0393045218,['Design'],A look at the incredibly well-preserved ancien...,1,design,look incredibly wellpreserved ancient mummies ...,167


In [ ]:
test = master_book_df['summary_clean'][0]

In [ ]:
!pip install lingua-language-detector

In [ ]:
from lingua import LanguageDetectorBuilder

detector = LanguageDetectorBuilder.from_all_languages().with_preloaded_language_models().build()
language = detector.detect_language_of("This is a test tweet in English.")
print(f"Detected language: {language}")

Detected language: Language.ENGLISH


In [ ]:
from lingua import LanguageDetectorBuilder, Language

detector = LanguageDetectorBuilder.from_all_languages().with_preloaded_language_models().build()

def detect_language(text):
    try:
        return detector.detect_language_of(text)
    except:
        return 'could not detect language'
# we are running the detect_language methon on the 'text_clean' column
master_book_df.loc[:,'lang'] = master_book_df.loc[:,'summary_clean'].apply(detect_language)

In [ ]:
master_book_df.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,isbn,Category,Summary,rating_count,cat_clean,summary_clean,len_sum,lang
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,0195153448,['Social Science'],Provides an introduction to classical myths pl...,1,social science,provides introduction classical myths placing ...,179,Language.ENGLISH
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,0002005018,['Actresses'],"In a small town in Canada, Clara Callan reluct...",14,actresses,small town canada clara callan reluctantly tak...,81,Language.ENGLISH
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,0060973129,['1940-1949'],"Here, for the first time in paperback, is an o...",3,1940-1949,first time paperback outstanding military hist...,171,Language.ENGLISH
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,0374157065,['Medical'],"Describes the great flu epidemic of 1918, an o...",11,medical,describes great flu epidemic outbreak killed f...,171,Language.ENGLISH
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton & Company,0393045218,['Design'],A look at the incredibly well-preserved ancien...,1,design,look incredibly wellpreserved ancient mummies ...,167,Language.ENGLISH


In [ ]:
# the count of english slightly changes every time I run the above method but the count should be around #61608
print(master_book_df.loc[:,'lang'].value_counts())

lang
Language.ENGLISH        131496
Language.TAGALOG          1496
Language.GERMAN           1031
Language.SPANISH           983
Language.LATIN             873
Language.FRENCH            635
Language.DUTCH             257
Language.WELSH             209
Language.ITALIAN           205
Language.DANISH            195
Language.PORTUGUESE        172
Language.BOKMAL            134
Language.YORUBA             77
Language.NYNORSK            74
Language.SWEDISH            66
Language.CATALAN            63
Language.AFRIKAANS          51
Language.BOSNIAN            51
Language.SOTHO              48
Language.TSONGA             37
Language.ESPERANTO          35
Language.ROMANIAN           34
Language.TSWANA             33
Language.SHONA              31
Language.POLISH             20
Language.SLOVAK             20
Language.HUNGARIAN          17
Language.ZULU               17
Language.XHOSA              16
Language.GANDA              15
Language.INDONESIAN         14
Language.MALAY              12
Lan

In [ ]:
# show non english rows
master_book_df = master_book_df[master_book_df['lang'] == Language.ENGLISH].reset_index(drop=True)
print(master_book_df.loc[:,'lang'].value_counts())


lang
Language.ENGLISH    131496
Name: count, dtype: int64


In [ ]:
len(master_book_df)

131496

**Make sure that I am not dropping to many because there are still duplicates**

In [ ]:
# record the sum of ratings for the duplicates so they are preserved when we drop duplicates
master_book_df['rating_count_total'] = master_book_df.groupby('Book-Title')['rating_count'].transform('sum')

In [ ]:
less_than_20_condition = master_book_df['rating_count_total'] < 20
master_book_df = master_book_df[~less_than_20_condition].reset_index(drop=True)

In [ ]:
master_book_df.drop(columns=['isbn', 'Category', 'Summary', 'len_sum', 'rating_count', 'lang'], inplace=True)
master_book_df = master_book_df.rename(columns={'cat_clean': 'Category', 'summary_clean': 'Summary', 'rating_count_total': 'rating_count'})

**There are still duplicates**

In [ ]:
dup_title = master_book_df[master_book_df[['Book-Title']].duplicated(keep=False)].sort_values(by=['Book-Title'])
print(len(dup_title))
print(dup_title.head())
print(dup_title[dup_title['Summary'].isna()])

4962
            ISBN      Book-Title    Book-Author  Year-Of-Publication  \
4864  0399143025  10 Lb. Penalty   Dick Francis                 1997   
5824  0515123471  10 Lb. Penalty   Dick Francis                 1998   
7834  0582060184            1984  George Orwell                 1991   
5382  0151660387            1984  George Orwell                 1983   
588   0451524934            1984  George Orwell                 1990   

                     Publisher                       Category  \
4864          Putnam Pub Group                        fiction   
5824                Jove Books  detective and mystery stories   
7834  Longman Publishing Group                satire, english   
5382                  Harcourt                    future life   
588                Signet Book               london (england)   

                                                Summary  rating_count  
4864  entering political arena trusted confidant pol...            61  
5824  ben juliards dream be

## **Transfer Book-Title to Ratings before Drop Title Duplicates with Multiple ISBNs**

In [ ]:
ratings.info()
ratings.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 3 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   User-ID      1149780 non-null  int64 
 1   ISBN         1149780 non-null  object
 2   Book-Rating  1149780 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 26.3+ MB


,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [ ]:
print(ratings.describe(include=['object']))

              ISBN
count      1149780
unique      340556
top     0971880107
freq          2502


In [ ]:
ratings.isnull().sum()

,0
User-ID,0
ISBN,0
Book-Rating,0


In [ ]:
ratings['ISBN'] = ratings['ISBN'].str.upper()

In [ ]:
master_book_df['ISBN'] = master_book_df['ISBN'].str.upper()

**Merge ratings with books and what to make sure all ratings have a title and summary**

In [ ]:
print(len(ratings))
ratings_df = pd.merge(ratings, master_book_df[['ISBN', 'Book-Title', 'Summary']], left_on='ISBN', right_on='ISBN', how='left')
print(len(ratings_df))

1149780
1149780


In [ ]:
print(ratings_df.head())
#print(ratings_check_isbn.describe(include=['object']))

   User-ID        ISBN  Book-Rating            Book-Title  \
0   276725  034545104X            0  Flesh Tones: A Novel   
1   276726  0155061224            5                   NaN   
2   276727  0446520802            0                   NaN   
3   276729  052165615X            3                   NaN   
4   276729  0521795028            6                   NaN   

                                             Summary  
0  twenty years affair legendary artist george ga...  
1                                                NaN  
2                                                NaN  
3                                                NaN  
4                                                NaN  


In [ ]:
print(len(ratings_df))
ratings_df = pd.merge(ratings_df, books[['ISBN', 'Book-Title']], left_on='ISBN', right_on='ISBN', how='left')
print(len(ratings_df))

1149780
1149780


In [ ]:
print(ratings_df.head())

   User-ID        ISBN  Book-Rating          Book-Title_x  \
0   276725  034545104X            0  Flesh Tones: A Novel   
1   276726  0155061224            5                   NaN   
2   276727  0446520802            0                   NaN   
3   276729  052165615X            3                   NaN   
4   276729  0521795028            6                   NaN   

                                             Summary  \
0  twenty years affair legendary artist george ga...   
1                                                NaN   
2                                                NaN   
3                                                NaN   
4                                                NaN   

                                        Book-Title_y  
0                               Flesh Tones: A Novel  
1                                   Rites of Passage  
2                                       The Notebook  
3                                     Help!: Level 1  
4  The Amsterdam Conn

In [ ]:
ratings_wo_content = ratings_df[ratings_df['Summary'].isnull()]
print(len(ratings_wo_content))

875718


In [ ]:
no_match_isbn = ratings_df[ratings_df['Book-Title_y'].isnull()]

118,605 is the difference between ratings.csv and combo.csv number of ratings

We are going to drop the ratings without matching isbn numbers

In [ ]:
len(no_match_isbn)

118605

In [ ]:
ratings_df.dropna(subset=['Book-Title_x'], inplace=True)
ratings_df.reset_index(drop=True, inplace=True)
print(len(ratings_df))

274062


In [ ]:
ratings_df.head()

,User-ID,ISBN,Book-Rating,Book-Title_x,Summary,Book-Title_y
0,276725,034545104X,0,Flesh Tones: A Novel,twenty years affair legendary artist george ga...,Flesh Tones: A Novel
1,276746,0449006522,0,Manhattan Hunt Club,college student jeff converse wrongly accused ...,Manhattan Hunt Club
2,276746,0553561618,0,Dark Paradise,court reporter marilee jennings embarks quest ...,Dark Paradise
3,276746,055356451X,0,Night Sins,gripping tale unfolds peaceful minnesota town ...,Night Sins
4,276747,0060517794,9,Little Altars Everywhere,companion beloved bestseller divine secrets ya...,Little Altars Everywhere


In [ ]:
ratings_df = ratings_df.rename(columns={'Book-Title_x': 'Book-Title'})

In [ ]:
ratings_df.drop(columns=['ISBN', 'Summary', 'Book-Title_y'], inplace=True)

In [ ]:
ratings_df.isna().sum()

,0
User-ID,0
Book-Rating,0
Book-Title,0


In [ ]:
len(ratings_df)

274062

In [ ]:
print(ratings_df.isnull().sum())
print(ratings_df.head())
print(ratings_df.shape)

User-ID        0
Book-Rating    0
Book-Title     0
dtype: int64
   User-ID  Book-Rating                Book-Title
0   276725            0      Flesh Tones: A Novel
1   276746            0       Manhattan Hunt Club
2   276746            0             Dark Paradise
3   276746            0                Night Sins
4   276747            9  Little Altars Everywhere
(274062, 3)


In [ ]:
# sort before drop duplicate to keep first publication
master_book_df.sort_values(by='Year-Of-Publication', inplace=True)
master_book_df.drop_duplicates(subset=['Book-Title'], inplace=True, keep='first')
master_book_df.reset_index(drop=True, inplace=True)

In [ ]:
print(master_book_df.isnull().sum())
print(master_book_df.head())
print(master_book_df.shape)

ISBN                   0
Book-Title             0
Book-Author            0
Year-Of-Publication    0
Publisher              0
Category               0
Summary                0
rating_count           0
dtype: int64
         ISBN                                 Book-Title          Book-Author  \
0  0590487922  Zlata's Diary: A Child's Life in Sarajevo      Zlata Filipovic   
1  0684174693                           Farewell to Arms     Ernest Hemingway   
2  0140250832                             House of Sleep         Jonathan Coe   
3  0571197639                Poisonwood Bible Edition Uk   Barbara Kingsolver   
4  1566190959                         The Scarlet Letter  Nathaniel Hawthorne   

   Year-Of-Publication             Publisher          Category  \
0                    0        Scholastic Inc  children and war   
1                    0  Simon Schuster Trade   tyrol (austria)   
2                    0  Penguin Putnam~trade  humorous stories   
3                    0       Faber F

## **Check Users**

In [ ]:
users.info()
users.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278858 entries, 0 to 278857
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   User-ID   278858 non-null  int64  
 1   Location  278858 non-null  object 
 2   Age       168096 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 6.4+ MB


,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [ ]:
users.isnull().sum()

,0
User-ID,0
Location,0
Age,110762


## **Save to csv**

In [ ]:
master_book_df.to_csv('/content/drive/MyDrive/Colab Notebooks/Machine Learning - Unsupervised/Final/Data/books_df.csv', index=False)

In [ ]:
ratings_df.to_csv('/content/drive/MyDrive/Colab Notebooks/Machine Learning - Unsupervised/Final/Data/ratings_df.csv', index=False)

In [ ]:
users.to_csv('/content/drive/MyDrive/Colab Notebooks/Machine Learning - Unsupervised/Final/Data/users_df.csv', index=False)